In [1]:
import sagemaker
import boto3
from sagemaker import get_execution_role

region = boto3.Session().region_name

session = sagemaker.Session()
bucket = session.default_bucket()
prefix = 'sagemaker/autopilot-ds4all/input'

role = get_execution_role()

sm = boto3.Session().client(service_name='sagemaker',region_name=region)

In [4]:
!wget -N https://sagemaker-ds4all.s3.us-east-2.amazonaws.com/Superstore.csv

--2020-04-14 20:26:46--  https://sagemaker-ds4all.s3.us-east-2.amazonaws.com/Superstore.csv
Resolving sagemaker-ds4all.s3.us-east-2.amazonaws.com (sagemaker-ds4all.s3.us-east-2.amazonaws.com)... 52.219.84.208
Connecting to sagemaker-ds4all.s3.us-east-2.amazonaws.com (sagemaker-ds4all.s3.us-east-2.amazonaws.com)|52.219.84.208|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘Superstore.csv’ not modified on server. Omitting download.



In [5]:
local_data_path = './Superstore.csv'

In [6]:
import pandas as pd

data = pd.read_csv(local_data_path, sep=',')
pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 10)         # Keep the output on one page
data

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,08/11/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,08/11/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,Hon Deluxe Fabric Upholstered Stacking Chairs....,731.9400,3,0.00,219.5820
2,3,CA-2016-138688,12/06/2016,16/06/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,11/10/2015,18/10/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,11/10/2015,18/10/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,9990,CA-2014-110422,21/01/2014,23/01/2014,Second Class,TB-21400,Tom Boeckenhauer,Consumer,United States,Miami,Florida,33180,South,FUR-FU-10001889,Furniture,Furnishings,Ultra Door Pull Handle,25.2480,3,0.20,4.1028
9990,9991,CA-2017-121258,26/02/2017,03/03/2017,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,California,92627,West,FUR-FU-10000747,Furniture,Furnishings,Tenex B1-RE Series Chair Mats for Low Pile Car...,91.9600,2,0.00,15.6332
9991,9992,CA-2017-121258,26/02/2017,03/03/2017,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,California,92627,West,TEC-PH-10003645,Technology,Phones,Aastra 57i VoIP phone,258.5760,2,0.20,19.3932
9992,9993,CA-2017-121258,26/02/2017,03/03/2017,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,California,92627,West,OFF-PA-10004041,Office Supplies,Paper,"It's Hot Message Books with Stickers. 2 3/4"" x 5""",29.6000,4,0.00,13.3200


In [7]:
import numpy as np

train_data, test_data, _ = np.split(data.sample(frac=1, random_state=123), 
                                                  [int(0.95 * len(data)), int(len(data))])  

In [8]:
# Save to CSV files
train_data.to_csv('train_data.csv', index=False, header=True, sep=',') # Need to keep column names
test_data.to_csv('test_data.csv', index=False, header=True, sep=',')

In [17]:
sess   = sagemaker.Session()

uri = sess.upload_data(path="train_data.csv", key_prefix=prefix)
print(uri)

In [18]:
#CRIAR MODELO VIA CÓDIGO
input_data_config = [{
      'DataSource': {
        'S3DataSource': {
          'S3DataType': 'S3Prefix',
          'S3Uri': 's3://{}/{}/train'.format(bucket,prefix)
        }
      },
      'TargetAttributeName': 'Profit'
    }
  ]

output_data_config = {
    'S3OutputPath': 's3://{}/{}/output'.format(bucket,prefix)
  }

In [19]:
from time import gmtime, strftime, sleep
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())

auto_ml_job_name = 'automl-ds4all-' + timestamp_suffix
print('AutoMLJobName: ' + auto_ml_job_name)

sm.create_auto_ml_job(AutoMLJobName=auto_ml_job_name,
                      InputDataConfig=input_data_config,
                      OutputDataConfig=output_data_config,
                      AutoMLJobConfig={'CompletionCriteria':
                                       {'MaxCandidates': 3}
                                      },
                      RoleArn=role)

AutoMLJobName: automl-ds4all-14-20-47-38


{'AutoMLJobArn': 'arn:aws:sagemaker:us-east-2:715254622466:automl-job/automl-ds4all-14-20-47-38',
 'ResponseMetadata': {'RequestId': 'd5e1f0c0-c16e-4a05-b7ac-95332e36e480',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd5e1f0c0-c16e-4a05-b7ac-95332e36e480',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '96',
   'date': 'Tue, 14 Apr 2020 20:47:37 GMT'},
  'RetryAttempts': 0}}

In [20]:
print ('JobStatus - Secondary Status')
print('------------------------------')


describe_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
print (describe_response['AutoMLJobStatus'] + " - " + describe_response['AutoMLJobSecondaryStatus'])
job_run_status = describe_response['AutoMLJobStatus']
    
while job_run_status not in ('Failed', 'Completed', 'Stopped'):
    describe_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    job_run_status = describe_response['AutoMLJobStatus']
    
    print (describe_response['AutoMLJobStatus'] + " - " + describe_response['AutoMLJobSecondaryStatus'])
    sleep(10)

In [21]:
best_candidate = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)['BestCandidate']
best_candidate_name = best_candidate['CandidateName']
print(best_candidate)
print('\n')
print("CandidateName: " + best_candidate_name)
print("FinalAutoMLJobObjectiveMetricName: " + best_candidate['FinalAutoMLJobObjectiveMetric']['MetricName'])
print("FinalAutoMLJobObjectiveMetricValue: " + str(best_candidate['FinalAutoMLJobObjectiveMetric']['Value']))

{'CandidateName': 'tuning-job-1-b34f9089cc8a462aa3-003-edb1709c', 'FinalAutoMLJobObjectiveMetric': {'MetricName': 'validation:mse', 'Value': 24089.80078125}, 'ObjectiveStatus': 'Succeeded', 'CandidateSteps': [{'CandidateStepType': 'AWS::SageMaker::ProcessingJob', 'CandidateStepArn': 'arn:aws:sagemaker:us-east-2:715254622466:processing-job/db-1-49ca43fce9774932a0b2a8d0d3e1c97c54621149f781480dab3444b346', 'CandidateStepName': 'db-1-49ca43fce9774932a0b2a8d0d3e1c97c54621149f781480dab3444b346'}, {'CandidateStepType': 'AWS::SageMaker::TrainingJob', 'CandidateStepArn': 'arn:aws:sagemaker:us-east-2:715254622466:training-job/automl-ds4-dpp0-1-896d924bda5a410ca7f7672c91f48295c89d7adbd1cf4', 'CandidateStepName': 'automl-ds4-dpp0-1-896d924bda5a410ca7f7672c91f48295c89d7adbd1cf4'}, {'CandidateStepType': 'AWS::SageMaker::TransformJob', 'CandidateStepArn': 'arn:aws:sagemaker:us-east-2:715254622466:transform-job/automl-ds4-dpp0-rpb-1-065092d827944718836f45b1b6efb97ece6f59ff8', 'CandidateStepName': 'aut

In [22]:
#CRIAR ENDPOINT VIA CÓDIGO

timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())
model_name = best_candidate_name + timestamp_suffix + "-model"
model_arn = sm.create_model(Containers=best_candidate['InferenceContainers'],
                            ModelName=model_name,
                            ExecutionRoleArn=role)

epc_name = best_candidate_name + timestamp_suffix + "-epc"
ep_config = sm.create_endpoint_config(EndpointConfigName = epc_name,
                                      ProductionVariants=[{'InstanceType': 'ml.t2.medium',
                                                           'InitialInstanceCount': 1,
                                                           'ModelName': model_name,
                                                           'VariantName': 'main'}])

ep_name = best_candidate_name + timestamp_suffix + "-ep"
create_endpoint_response = sm.create_endpoint(EndpointName=ep_name,
                                              EndpointConfigName=epc_name)

In [24]:
sm.get_waiter('endpoint_in_service').wait(EndpointName=ep_name)

In [ ]:
#teste de previsão com endpoint do melhor modelo
ep_name 

In [26]:
from io import StringIO
from sagemaker.predictor import RealTimePredictor
from sagemaker.content_types import CONTENT_TYPE_CSV


predictor = RealTimePredictor(
    endpoint=ep_name,
    sagemaker_session=session,
    content_type=CONTENT_TYPE_CSV,
    accept=CONTENT_TYPE_CSV)

# Remove the target column from the test data
test_data_inference = test_data.drop('Profit', axis=1)

# Obtain predictions from SageMaker endpoint
prediction = predictor.predict(test_data_inference.to_csv(sep=',', header=False, index=False)).decode('utf-8')

# Load prediction in pandas and compare to ground truth
prediction_df = pd.read_csv(StringIO(prediction), header=None)

#accuracy = (test_data.reset_index()['Profit'] == prediction_df[0]).sum() / len(test_data_inference)
#print('Accuracy: {}'.format(accuracy))

In [27]:
prediction_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499
0,5.106828,3.891559,6.204206,-3.425479,51.916355,3.845194,5.6827,4.414884,1.103909,10.229033,8.210451,16.646925,93.387466,3.634101,4.10646,-13.092308,-35.973827,3.500717,15.050244,1.820268,30.391064,22.440525,-52.197998,9.796835,55.750027,19.674726,5.093165,105.068153,10.651158,1.058732,319.89856,41.069191,5.624402,15.477948,6.315346,6.226402,8.246201,6.716605,17.926346,3.849658,45.446186,14.742238,4.428581,50.89399,3.749109,9.318865,14.137471,6.258669,1.147483,1.372181,4.507153,39.032841,5.446502,10.446441,86.479973,17.302979,8.265939,60.854294,21.017719,-6.571764,5.060173,102.81115,0.917667,12.840517,8.029594,9.312609,25.804329,16.393826,47.104561,49.346272,-10.239359,6.155863,-8.950932,13.353403,-0.221971,2.801897,-3.086722,36.260487,12.853203,21.972055,3.883512,-3.763637,60.059666,4.297585,42.441494,-33.208694,16.495167,-193.48967,22.371151,9.194493,9.129716,28.388248,5.303662,10.651158,1.820268,-14.400347,-13.401252,10.112587,10.053125,111.178215,-0.945754,116.996719,3.770495,-139.31456,33.199249,2.569792,23.900578,2.895101,10.611403,8.395138,-5.164294,-5.998645,23.398409,6.62567,129.521698,8.433462,6.55571,3.693841,1.529402,3.815068,3.820411,19.963469,7.346687,7.326012,2.926835,2.432517,1.5618,-1.315127,-7.849216,103.795883,25.496525,3.165248,7.959908,74.871582,3.430492,7.14659,-39.218967,-0.867833,-2.357723,-13.430566,11.077395,14.223056,14.950843,4.573588,18.503325,4.360997,6.321498,1.449322,7.681576,-116.593452,4.850513,425.620789,0.461399,-8.385652,5.396521,4.422297,-5.159439,7.681105,1.151737,0.934499,5.380596,1.851016,19.346468,12.252287,5.145375,2.455732,17.910322,-1.647089,-13.663659,30.329237,33.262672,39.221783,-17.338554,18.127584,15.35557,-10.502511,6.257378,15.856547,2.046092,4.105692,3.773898,4.675606,7.00225,-13.111835,2.198709,-3.769664,13.468906,12.11046,23.170202,10.62116,3.51032,8.902404,56.832657,15.996353,322.709808,52.879444,17.044613,12.901205,2.70529,5.145375,18.171534,1.141839,7.266524,-6.753064,57.774822,4.592771,21.344273,1.725761,4.83125,140.396149,-36.418495,-82.493408,11.115165,1.520798,8.420334,3.821291,9.312609,127.237511,18.741072,4.0

In [28]:
#CLEAN UP

#s3 = boto3.resource('s3')
#s3_bucket = s3.Bucket(bucket)

#job_outputs_prefix = '{}/output/{}'.format(prefix, auto_ml_job_name)
#s3_bucket.objects.filter(Prefix=job_outputs_prefix).delete()
#sm.delete_endpoint(EndpointName=ep_name)
#sm.delete_endpoint_config(EndpointConfigName=epc_name)
#sm.delete_model(ModelName=model_name)